In [3]:
import pandas as pd
import mysql.connector

conn = mysql.connector.connect(
    host = "localhost",
    user="StockInsertion",
    password="StockDataInsertion",
    database="Stock"
)
if conn.is_connected():
    print("Connected")

cursor = conn.cursor()

Connected


In [4]:
print(pd.read_excel('StockData/GPW/_2022-03-03_akcje.xls').head(1))

         Data    Nazwa          ISIN Waluta  Kurs otwarcia  Kurs max  \
0  2022-03-03  06MAGNA  PLNFI0600010    PLN          2.885     2.885   

   Kurs min  Kurs zamknięcia  Zmiana  Wolumen  Liczba Transakcji   Obrót  \
0      2.62             2.74   -2.32    42279                 98  115.17   

   Liczba otwartych pozycji  Wartość otwartych pozycji  Cena nominalna  
0                         0                          0               0  


# Prepare the list of companies on GPW

In [ ]:
import os
ExchangePath = 'StockData/GPW'
Companies = set()
for CurrentFile in os.listdir(ExchangePath):
    file_path = os.path.join(ExchangePath, CurrentFile)
    df = pd.read_excel(file_path)

    if 'Nazwa' in df.columns:
        for company in df['Nazwa']:
            Companies.add(company)
pd.DataFrame(sorted(Companies), columns=['company']).to_csv("GPW_Companies.csv")


In [ ]:
cursor.close()
conn.close()

In [7]:
import mysql.connector
from mysql.connector import Error

def insert_stock_exchange():
    try:
        conn = mysql.connector.connect(
            host="localhost",
            user="root",       # replace with your MySQL user
            password="Kanakan9824",   # replace with your password
            database="Stock"    # replace with your DB name (e.g., Stock)
        )
        if conn.is_connected():
            print("Connected to database")

        cursor = conn.cursor()

        insert_query = """
            INSERT INTO StockExchange (ExchangeName, Country, City)
            VALUES (%s, %s, %s)
        """

        data = ("GPW", "Polska", "Warszawa")

        cursor.execute(insert_query, data)
        conn.commit()
        print("Record inserted successfully")

    except Error as e:
        print(f"Error: {e}")

    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            print("MySQL connection closed")

if __name__ == "__main__":
    insert_stock_exchange()


Connected to database
Record inserted successfully
MySQL connection closed
